EAR/MAR

In [13]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
import csv
from imutils import face_utils
import os

# Initialize dlib's face detector and landmark predictor
predictor_path = r"F:\SKRIPSI\TA\Kartivel\Driver-Drowsiness-Detection-main\shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])  # 65, 59
    B = dist.euclidean(mouth[12], mouth[16])  # 63, 57
    C = dist.euclidean(mouth[0], mouth[6])    # 49, 55
    return (A + B) / (2.0 * C)

def process_videos_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith((".mp4",".mov",".MOV")):
            video_path = os.path.join(folder_path, filename)
            process_video(video_path)

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    video_title = os.path.splitext(os.path.basename(video_path))[0]
    csv_filename = f"{video_title}_data_analysis.csv"
    data_list = []

    # Get the video's frames per second (fps)
    fps = cap.get(cv2.CAP_PROP_FPS)
    delay = int(100 / fps)  # Calculate delay in milliseconds

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if faces:
            for face in faces:
                shape = predictor(gray, face)
                landmarks = face_utils.shape_to_np(shape)
                ear = 0
                mar = 0

                if len(landmarks) >= 68:
                    leftEye = landmarks[36:42]
                    rightEye = landmarks[42:48]
                    mouth = landmarks[48:68]

                    ear = (eye_aspect_ratio(leftEye) + eye_aspect_ratio(rightEye)) / 2.0
                    mar = mouth_aspect_ratio(mouth)

                    cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
                    cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
                    cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)

                    cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f"MAR: {mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

                frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                data_list.append([frame_number, ear, mar])

        else:
            # No faces detected
            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            data_list.append([frame_number, 0, 0])

        cv2.imshow("Frame", frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    with open(csv_filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Frame', 'EAR', 'MAR'])
        csvwriter.writerows(data_list)
        print(f"Data written to {csv_filename}")

video_folder_path = r"F:\SKRIPSI\TA\UTA-RLDD Dataset\3\3Video\New_folder"
process_videos_in_folder(video_folder_path)

Data written to 2_3_data_analysis.csv
Data written to 3_2_data_analysis.csv


OPENESS

In [1]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
import csv
from imutils import face_utils
import os

#inisialisasi prediktor dlib
predictor_path = r"F:\SKRIPSI\TA\Kartivel\Driver-Drowsiness-Detection-main\shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])  # 65, 59
    B = dist.euclidean(mouth[12], mouth[16])  # 63, 57
    C = dist.euclidean(mouth[0], mouth[6])   # 49, 55
    return (A + B) / (2.0 * C)

def calculate_eye_openness(eye_image):
    """ Calculate the openness of the eye from the binary image """
    white_pixels = np.sum(eye_image == 255)
    total_pixels = eye_image.size
    return white_pixels / total_pixels if total_pixels > 0 else 0

def get_eye_image(frame, eye_points):
    """ Extract and return a binary image of the eye """
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [eye_points], 255)
    eye = cv2.bitwise_and(frame, frame, mask=mask)
    (x, y, w, h) = cv2.boundingRect(eye_points)
    eye = eye[y:y+h, x:x+w]
    gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    _, binary_eye = cv2.threshold(gray_eye, 30, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_eye

def process_videos_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(folder_path, filename)
            process_video(video_path)

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    video_title = os.path.splitext(os.path.basename(video_path))[0]
    csv_filename = f"{video_title}_data_analysis.csv"
    data_list = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray, 0)

        if faces:
            for face in faces:
                shape = predictor(gray, face)
                landmarks = face_utils.shape_to_np(shape)
                #ear = 0
                mar =0

                if len(landmarks) >= 48:
                    leftEye = landmarks[36:42]
                    rightEye = landmarks[42:48]
                    mouth = landmarks[48:68]
                    left_eye_image = get_eye_image(frame, leftEye)
                    right_eye_image = get_eye_image(frame, rightEye)

                    left_eye_openness = calculate_eye_openness(left_eye_image)
                    right_eye_openness = calculate_eye_openness(right_eye_image)
                    eye_openness = ((left_eye_openness + right_eye_openness) / 2 ) - 0.2
                    mar = mouth_aspect_ratio(mouth)
                    #ear = eye_aspect_ratio()

                    cv2.drawContours(frame, [cv2.convexHull(leftEye)], -1, (0, 255, 0), 1)
                    cv2.drawContours(frame, [cv2.convexHull(rightEye)], -1, (0, 255, 0), 1)
                    cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)

                    cv2.imshow("Left Eye Binary", left_eye_image)
                    cv2.imshow("Right Eye Binary", right_eye_image)
                    cv2.putText(frame, f"Openness: {eye_openness:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    #cv2.putText(frame, f"Openness: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f"MAR: {mar:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    eye_openness =((left_eye_openness + right_eye_openness) / 2) - 0.2
                    #ear = 0
                    mar = 0

                cv2.imshow("Frame", frame)
                frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                video_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 
                data_list.append([frame_number, video_time, eye_openness, mar])

        else:
            # No faces detected
            frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            video_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 
            data_list.append([frame_number, video_time, 0, 0]) 

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    with open(csv_filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Frame', 'time', 'Openness', 'MAR'])
        csvwriter.writerows(data_list)
        print(f"Data written to {csv_filename}")


video_folder_path = r"F:\SKRIPSI\TA\DROZY\videos_i8\real"
process_videos_in_folder(video_folder_path)


Data written to 1-1_data_analysis.csv
Data written to 1-2_data_analysis.csv
Data written to 1-3_data_analysis.csv
Data written to 10-1_data_analysis.csv
Data written to 10-3_data_analysis.csv
Data written to 11-1_data_analysis.csv
Data written to 11-2_data_analysis.csv
Data written to 11-3_data_analysis.csv
Data written to 12-1_data_analysis.csv
Data written to 13-1_data_analysis.csv
Data written to 13-2_data_analysis.csv
Data written to 14-1_data_analysis.csv
Data written to 14-2_data_analysis.csv
Data written to 14-3_data_analysis.csv
Data written to 2-1_data_analysis.csv
Data written to 2-2_data_analysis.csv
Data written to 2-3_data_analysis.csv
Data written to 3-1_data_analysis.csv
Data written to 3-2_data_analysis.csv
Data written to 3-3_data_analysis.csv
Data written to 4-1_data_analysis.csv
Data written to 4-2_data_analysis.csv
Data written to 4-3_data_analysis.csv
Data written to 5-1_data_analysis.csv
Data written to 5-2_data_analysis.csv
Data written to 5-3_data_analysis.csv
D

INTERPOLASI 1

In [14]:
import pandas as pd
import numpy as np
import os

# Set your directory path where CSV files are stored
directory_path = r'F:\SKRIPSI\TA\Fix_code\SkripSHYs\FixedDataset\NonAugmented\test'  # Replace with your directory path

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    # Check for CSV files
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Replace 0.0 with NaN for interpolation in 'Openness' and 'MAR' columns
        df['EAR'] = df['EAR'].replace(0.0, np.nan)
        df['MAR'] = df['MAR'].replace(0.0, np.nan)
        
        # Use linear interpolation to fill NaNs in both columns
        df['EAR'].interpolate(method='linear', inplace=True)
        df['MAR'].interpolate(method='linear', inplace=True)
        
        # After interpolation, if you need to round the values or process them in some way, do it here
        
        # Save the updated DataFrame back to its CSV file
        df.to_csv(file_path, index=False)  # Save the file with interpolated data

print("Interpolation complete for all CSV files.")

Interpolation complete for all CSV files.


Interpolasi per file

In [4]:
import pandas as pd
import numpy as np
import os

# Define the input file path and output file path
input_file_path = r'F:\SKRIPSI\TA\Fix_code\SkripSHYs\PERCLOS\1_2_data_analysis.csv'  # Replace with your input file path
output_file_path = r'F:\SKRIPSI\TA\Fix_code\SkripSHYs\PERCLOS\1-2_final_output.csv'  # Replace with your output file path

# Load the CSV file into a DataFrame
df = pd.read_csv(input_file_path)

# Replace 0.0 with NaN for interpolation in 'EAR' and 'MAR' columns
df['EAR'] = df['EAR'].replace(0.0, np.nan)
df['MAR'] = df['MAR'].replace(0.0, np.nan)

# Use linear interpolation to fill NaNs in both columns
df['EAR'].interpolate(method='linear', inplace=True)
df['MAR'].interpolate(method='linear', inplace=True)

# After interpolation, if you need to round the values or process them in some way, do it here

# Save the updated DataFrame to the output CSV file
df.to_csv(output_file_path, index=False)  # Save the file with interpolated data

print("Interpolation complete for the CSV file.")


Interpolation complete for the CSV file.


Sorting data

In [40]:
import pandas as pd

csv_filepath = r'F:\SKRIPSI\TA\Fix_code\SkripSHYs\FixedDataset\NonAugmented\test\1_2_data_analysis.csv'

def calculate_threshold(csv_filepath):
    data = pd.read_csv(csv_filepath)

    # Hitung nilai 1/20 total data
    total_frames = len(data)
    A = total_frames // 20
    A = max(A, 1)

    sorted_data = data.sort_values(by='EAR', ascending=False)

    # hitung O and C untuk treshold
    O = sorted_data['EAR'].head(A).mean()
    C = sorted_data['EAR'].tail(A).mean()

    # Hitung Treshold
    threshold = (O - C) * 0.2 + C

    print("Total frames:", total_frames)
    print("A (1/20 of total frames):", A)
    print("O (Open eyes state mean):", O)
    print("C (Closed eyes state mean):", C)
    print("Threshold:", threshold)
    print (sorted_data['EAR'].tail(A))
    
    return threshold

threshold = calculate_threshold(csv_filepath)


Total frames: 18263
A (1/20 of total frames): 913
O (Open eyes state mean): 0.3443858373539099
C (Closed eyes state mean): 0.22481621155795736
Threshold: 0.24873013671714786
8415     0.260008
14636    0.259980
9963     0.259946
8269     0.259887
8416     0.259857
           ...   
15695    0.107453
15683    0.106405
14666    0.106226
6988     0.105514
15682    0.090235
Name: EAR, Length: 913, dtype: float64


COBA COBA PERCLOS

In [41]:
import pandas as pd

# Fungsi untuk memproses data per 30 baris dan mengelompokkan berdasarkan threshold
def process_data(file_path, threshold):
    df = pd.read_csv(file_path)
    if 'EAR' not in df.columns or 'MAR' not in df.columns:
        raise ValueError("Kolom 'EAR' atau 'MAR' tidak ditemukan dalam file CSV")
    
    result = []

    # Pilah data per 30 baris
    for i in range(0, len(df), 30):
        chunk_openness = df['EAR'].iloc[i:i+30]
        chunk_mar = df['MAR'].iloc[i:i+30]

        # Pisahkan data di atas dan di bawah threshold
        above_threshold = chunk_openness[chunk_openness > threshold]
        below_threshold = chunk_openness[chunk_openness <= threshold]
        
        # Hitung jumlah masing-masing kelompok
        sum_above = above_threshold.sum()
        sum_below = below_threshold.sum()

        # Hitung standar deviasi MAR untuk setiap chunk
        std_mar = chunk_mar.std()
        
        result.append({
            'Sum_Above_Threshold': sum_above,
            'Sum_Below_Threshold': sum_below,
            'MAR_Std': std_mar
        })

    # Buat DataFrame dari hasil
    result_df = pd.DataFrame(result)
    
    return result_df

# Fungsi untuk melakukan perhitungan rasio dan hanya memasukkan nilai di atas 0
def calculate_ratio(data):
    data['PERCLOS'] = data['Sum_Below_Threshold'] / (data['Sum_Above_Threshold'] + data['Sum_Below_Threshold'])
    
    # Hanya simpan baris dengan nilai PERCLOS di atas 0
    result_df = data[(data['PERCLOS'] > 0) & (data['PERCLOS'] < 0.8)]
    result_df = result_df[['PERCLOS', 'MAR_Std']]
    
    return result_df

file_path = r'F:\SKRIPSI\TA\Fix_code\SkripSHYs\FixedDataset\NonAugmented\test\1_2_data_analysis.csv' 
Threshold = threshold  # Ganti dengan nilai threshold yang diinginkan
final_output_path = r'F:\SKRIPSI\TA\Fix_code\SkripSHYs\FixedDataset\NonAugmented\test\1_2_ready_output.csv'

processed_data = process_data(file_path, Threshold)
final_result = calculate_ratio(processed_data)
final_result.to_csv(final_output_path, index=False)
print(threshold)


0.24873013671714786
